In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense
import numpy as np

In [4]:
df = pd.read_csv(r'C:\Users\pinkpigma\pinkpigma的同步盘\KDD研二上\可解释性框架-工作\数据\patient_diagnoses2000_ndc300_delete1.csv',)


In [5]:
x = df.iloc[:,3:3+71+2000]
y = df.iloc[:,3+71+2000:]
from sklearn.impute import SimpleImputer

from sklearn. preprocessing import LabelEncoder 
x['gender'] = LabelEncoder().fit_transform(x['gender'])
x['ethnicity'] = LabelEncoder().fit_transform(x['ethnicity'])
x['ethnicity_grouped'] = LabelEncoder().fit_transform(x['ethnicity_grouped'])
x['first_hosp_stay'] = LabelEncoder().fit_transform(x['first_hosp_stay'])

imp = SimpleImputer(missing_values=np.nan, strategy='mean')

x = imp.fit_transform(x)



In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
from keras import backend as K
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# y_train

In [ ]:
from keras import backend as K
from keras import losses
import numpy as np
import tensorflow as tf
def Jaccard_Loss(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    y_true_expand = K.expand_dims(y_true, axis=0)
    y_pred_expand = K.expand_dims(y_pred, axis=-1)

    fenzi = K.dot(y_true_expand, y_pred_expand)

    fenmu_1 = K.sum(y_true, keepdims=True)

    fenmu_2 = K.ones_like(y_true_expand) - y_true_expand
    fenmu_2 = K.dot(fenmu_2, y_pred_expand)

    return K.mean((tf.constant([[1]], dtype=tf.float32) - (fenzi / (fenmu_1 + fenmu_2))), axis=-1)

def JI(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    threshold_value = 0.3

    y_pred = K.cast(K.greater(y_pred, threshold_value), K.floatx())
    fenzi = K.sum(y_true * y_pred, keepdims=True)
    # true_positives_sum = K.sum(true_positives, keepdims=True)
    fenmu = K.sum(K.cast((K.greater(y_true + y_pred, 0.8)), K.floatx()), keepdims=True)

    return K.mean(fenzi / fenmu, axis=-1)



def deep_model(feature_dim,label_dim):
    from keras.models import Sequential
    from keras.layers import Dense
    model = Sequential()
    print("create model. feature_dim ={}, label_dim ={}".format(feature_dim, label_dim))
    model.add(Dense(512, activation='relu', input_dim=feature_dim))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(label_dim, activation='sigmoid'))
    model.compile(optimizer='adam', loss=Jaccard_Loss, metrics=[JI])
    return model

In [29]:
def train_deep(X_train,y_train,X_test,y_test):
    feature_dim = X_train.shape[1]
    label_dim = y_train.shape[1]
    model = deep_model(feature_dim,label_dim)
    model.summary()
    model.fit(X_train,y_train,batch_size=256, epochs=1000,validation_data=(X_test,y_test))

In [31]:
train_deep(X_train,y_train,X_test,y_test)

create model. feature_dim =2071, label_dim =300
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 512)               1060864   
                                                                 
 dense_37 (Dense)            (None, 512)               262656    
                                                                 
 dense_38 (Dense)            (None, 300)               153900    
                                                                 
Total params: 1,477,420
Trainable params: 1,477,420
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
120/120 [==============================] - 2s 15ms/step - loss: 0.4062 - JI: 0.0328 - val_loss: 0.1187 - val_JI: 0.0285
Epoch 2/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.1293 - JI: 0.0310 - val_loss: 0.1203 - val_JI: 

Epoch 56/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0906 - JI: 0.1266 - val_loss: 0.0937 - val_JI: 0.1157
Epoch 57/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0901 - JI: 0.1289 - val_loss: 0.0937 - val_JI: 0.1276
Epoch 58/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0902 - JI: 0.1293 - val_loss: 0.0940 - val_JI: 0.1185
Epoch 59/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0896 - JI: 0.1326 - val_loss: 0.0933 - val_JI: 0.1225
Epoch 60/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0895 - JI: 0.1331 - val_loss: 0.0938 - val_JI: 0.1154
Epoch 61/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0893 - JI: 0.1355 - val_loss: 0.0949 - val_JI: 0.1161
Epoch 62/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0894 - JI: 0.1362 - val_loss: 0.0942 - val_JI: 0.1241
Epoch 63/1000
120/120 [==============================] - 2s 14

120/120 [==============================] - 2s 14ms/step - loss: 0.0839 - JI: 0.1794 - val_loss: 0.0950 - val_JI: 0.1263
Epoch 118/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0833 - JI: 0.1837 - val_loss: 0.0955 - val_JI: 0.1231
Epoch 119/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0830 - JI: 0.1848 - val_loss: 0.0960 - val_JI: 0.1353
Epoch 120/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0829 - JI: 0.1863 - val_loss: 0.0963 - val_JI: 0.1384
Epoch 121/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0836 - JI: 0.1844 - val_loss: 0.0956 - val_JI: 0.1430
Epoch 122/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0960 - JI: 0.1838 - val_loss: 0.0960 - val_JI: 0.1335
Epoch 123/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0840 - JI: 0.1832 - val_loss: 0.0961 - val_JI: 0.1481
Epoch 124/1000
120/120 [==============================] - 2s 14ms/step

Epoch 178/1000
120/120 [==============================] - 2s 15ms/step - loss: 0.0803 - JI: 0.2151 - val_loss: 0.1025 - val_JI: 0.1435
Epoch 179/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0818 - JI: 0.2144 - val_loss: 0.1030 - val_JI: 0.1255
Epoch 180/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0800 - JI: 0.2151 - val_loss: 0.1027 - val_JI: 0.1399
Epoch 181/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0794 - JI: 0.2177 - val_loss: 0.1025 - val_JI: 0.1396
Epoch 182/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0799 - JI: 0.2162 - val_loss: 0.1021 - val_JI: 0.1382
Epoch 183/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0800 - JI: 0.2139 - val_loss: 0.1031 - val_JI: 0.1408
Epoch 184/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0783 - JI: 0.2244 - val_loss: 0.1019 - val_JI: 0.1350
Epoch 185/1000
120/120 [==============================]

120/120 [==============================] - 2s 13ms/step - loss: 0.0743 - JI: 0.2563 - val_loss: 0.1093 - val_JI: 0.1445
Epoch 239/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0747 - JI: 0.2538 - val_loss: 0.1104 - val_JI: 0.1446
Epoch 240/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0750 - JI: 0.2522 - val_loss: 0.1121 - val_JI: 0.1383
Epoch 241/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0751 - JI: 0.2514 - val_loss: 0.1119 - val_JI: 0.1425
Epoch 242/1000
120/120 [==============================] - 2s 14ms/step - loss: 0.0745 - JI: 0.2555 - val_loss: 0.1104 - val_JI: 0.1443
Epoch 243/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0747 - JI: 0.2537 - val_loss: 0.1110 - val_JI: 0.1399
Epoch 244/1000
120/120 [==============================] - 2s 13ms/step - loss: 0.0744 - JI: 0.2573 - val_loss: 0.1119 - val_JI: 0.1382
Epoch 245/1000
120/120 [==============================] - 2s 13ms/step

...

120/120 [==============================] - 3s 24ms/step - loss: 0.0714 - JI: 0.2880 - val_loss: 0.1582 - val_JI: 0.1365
Epoch 723/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0696 - JI: 0.2990 - val_loss: 0.1607 - val_JI: 0.1285
Epoch 724/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0666 - JI: 0.3167 - val_loss: 0.1654 - val_JI: 0.1308
Epoch 725/1000
120/120 [==============================] - 3s 26ms/step - loss: 0.0659 - JI: 0.3231 - val_loss: 0.1609 - val_JI: 0.1295
Epoch 726/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0655 - JI: 0.3264 - val_loss: 0.1641 - val_JI: 0.1287
Epoch 727/1000
120/120 [==============================] - 3s 25ms/step - loss: 0.0654 - JI: 0.3279 - val_loss: 0.1625 - val_JI: 0.1257
Epoch 728/1000
120/120 [==============================] - 3s 25ms/step - loss: 0.0660 - JI: 0.3230 - val_loss: 0.1635 - val_JI: 0.1303
Epoch 729/1000
120/120 [==============================] - 3s 25ms/step

Epoch 783/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0637 - JI: 0.3408 - val_loss: 0.1688 - val_JI: 0.1227
Epoch 784/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0642 - JI: 0.3370 - val_loss: 0.1717 - val_JI: 0.1300
Epoch 785/1000
120/120 [==============================] - 3s 25ms/step - loss: 0.0640 - JI: 0.3383 - val_loss: 0.1671 - val_JI: 0.1274
Epoch 786/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0643 - JI: 0.3367 - val_loss: 0.1752 - val_JI: 0.1272
Epoch 787/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0647 - JI: 0.3334 - val_loss: 0.1721 - val_JI: 0.1319
Epoch 788/1000
120/120 [==============================] - 3s 22ms/step - loss: 0.0640 - JI: 0.3388 - val_loss: 0.1685 - val_JI: 0.1246
Epoch 789/1000
120/120 [==============================] - 3s 22ms/step - loss: 0.0641 - JI: 0.3382 - val_loss: 0.1701 - val_JI: 0.1225
Epoch 790/1000
120/120 [==============================]

120/120 [==============================] - 3s 24ms/step - loss: 0.0640 - JI: 0.3393 - val_loss: 0.1794 - val_JI: 0.1203
Epoch 844/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0640 - JI: 0.3386 - val_loss: 0.1743 - val_JI: 0.1290
Epoch 845/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0639 - JI: 0.3396 - val_loss: 0.1749 - val_JI: 0.1272
Epoch 846/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0747 - JI: 0.2809 - val_loss: 0.1636 - val_JI: 0.1357
Epoch 847/1000
120/120 [==============================] - 3s 24ms/step - loss: 0.0792 - JI: 0.2549 - val_loss: 0.1694 - val_JI: 0.1301
Epoch 848/1000
120/120 [==============================] - 3s 26ms/step - loss: 0.0750 - JI: 0.2961 - val_loss: 0.1680 - val_JI: 0.1316
Epoch 849/1000
120/120 [==============================] - 3s 26ms/step - loss: 0.0797 - JI: 0.2706 - val_loss: 0.1621 - val_JI: 0.1226
Epoch 850/1000
120/120 [==============================] - 3s 25ms/step

Epoch 904/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0636 - JI: 0.3419 - val_loss: 0.1816 - val_JI: 0.1286
Epoch 905/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0631 - JI: 0.3457 - val_loss: 0.1803 - val_JI: 0.1280
Epoch 906/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0630 - JI: 0.3472 - val_loss: 0.1815 - val_JI: 0.1258
Epoch 907/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0667 - JI: 0.3242 - val_loss: 0.1789 - val_JI: 0.1262
Epoch 908/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0661 - JI: 0.3237 - val_loss: 0.1826 - val_JI: 0.1269
Epoch 909/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0754 - JI: 0.2843 - val_loss: 0.1541 - val_JI: 0.1072
Epoch 910/1000
120/120 [==============================] - 3s 22ms/step - loss: 0.0765 - JI: 0.2676 - val_loss: 0.1721 - val_JI: 0.1228
Epoch 911/1000
120/120 [==============================]

120/120 [==============================] - 3s 23ms/step - loss: 0.0631 - JI: 0.3456 - val_loss: 0.1811 - val_JI: 0.1269
Epoch 965/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0627 - JI: 0.3492 - val_loss: 0.1839 - val_JI: 0.1235
Epoch 966/1000
120/120 [==============================] - 3s 22ms/step - loss: 0.0641 - JI: 0.3391 - val_loss: 0.1848 - val_JI: 0.1227
Epoch 967/1000
120/120 [==============================] - 3s 22ms/step - loss: 0.0631 - JI: 0.3459 - val_loss: 0.1857 - val_JI: 0.1246
Epoch 968/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0633 - JI: 0.3448 - val_loss: 0.1837 - val_JI: 0.1251
Epoch 969/1000
120/120 [==============================] - 3s 23ms/step - loss: 0.0631 - JI: 0.3459 - val_loss: 0.1863 - val_JI: 0.1223
Epoch 970/1000
120/120 [==============================] - 3s 22ms/step - loss: 0.0655 - JI: 0.3318 - val_loss: 0.1808 - val_JI: 0.1312
Epoch 971/1000
120/120 [==============================] - 3s 23ms/step

In [32]:
import shap